# 📊 Dataset Maker by Hollowstrawberry

Here is a [guide to using this colab](https://civitai.com/models/22530). It will help you make a dataset quickly and using it to train a Lora.

This is based on the work of [Kohya-ss and Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr). Image curation adapted from [this other colab](https://colab.research.google.com/drive/1oBSntB40BKzNmKceXUlkXzujzdQw-Ci7#scrollTo=fl4E_J5g2grT). Thank you!

|Colab|English|Spanish|
|:--|:-:|:-:|
| 📊 **Dataset Maker** | <a target="_blank" href="https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a> | [🇪🇸](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | <a target="_blank" href="https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a> | [🇪🇸](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |

In [ ]:
#@title ## 🚩 Start Here

import os
import shutil
import zipfile
from IPython.utils import capture
from google.colab import drive

#@markdown ### 1️⃣ Setup
#@markdown This cell will connect to your Google Drive and create the necessary folders under `lora_training`. <p>
#@markdown Your project name will be the same as the folder containing your images. Spaces aren't allowed.
project_name = "" #@param {type:"string"}
project_name = project_name.strip()

if not project_name or " " in project_name:
  print("Please write a valid project_name.")
else:
  if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  #root_dir
  root_dir = "/content"
  deps_dir = os.path.join(root_dir,"deps")
  main_dir = os.path.join(root_dir,"drive/MyDrive/lora_training")
  datasets_dir = os.path.join(main_dir,"datasets")
  config_dir = os.path.join(main_dir,"config")
  images_folder = os.path.join(datasets_dir, project_name)
  config_folder = os.path.join(config_dir, project_name)

  for dir in [deps_dir, main_dir, datasets_dir, config_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ Project {project_name} is ready!")

In [ ]:
#@markdown ### 2️⃣ Scrape images from Gelbooru

import os
import html
import json
import time
from urllib.request import urlopen, Request
from IPython.display import Markdown, display
from google.colab import output as console

#@markdown We will grab images from the popular anime gallery [Gelbooru](https://gelbooru.com/). Images are sorted by tags, including poses, scenes, character traits, character names, artists, etc. <p>
#@markdown * If you instead want to use your own images, upload them to your Google Drive's `lora_training/datasets/project_name` folder.
#@markdown * If you instead want to download screencaps of anime episodes, try [this other colab by another person](https://colab.research.google.com/drive/1oBSntB40BKzNmKceXUlkXzujzdQw-Ci7). It's more complicated though.

#@markdown Up to 1000 images may be downloaded by this step in just one minute. Don't abuse it. <p>
#@markdown Your target tags should probably include a minimum score, exclude explicit images (they often make training harder) and include the relevant tags for your character/concept/artstyle.
#@markdown Separate words with underscores, separate tags with spaces, and use - to exclude a tag.
tags = "score:>10 -rating:explicit -loli -futanari -monochrome 1girl solo rem_(re:zero)" #@param {type:"string"}
##@markdown If an image is bigger than this resolution a smaller version will be downloaded instead.
max_resolution = 2048 #param {type:"slider", min:2048, max:8196, step:2048}
##@markdown Posts with a parent post are often minor variations of the same image.
include_posts_with_parent = True #param {type:"boolean"}

tags = tags.replace(" ", "+")\
           .replace("(", "%28")\
           .replace(")", "%29")\
           .replace(":", "%3a")\
          
url = "https://gelbooru.com/index.php?page=dapi&json=1&s=post&q=index&limit=100&tags={}".format(tags)
user_agent = "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/93.0.4577.83 Safari/537.36"
limit = 100 # hardcoded by gelbooru
total_limit = 1000 # you can edit this if you want but I wouldn't recommend it
supported_types = (".png", ".jpg", ".jpeg")

def ubuntu_deps(url, name, dst):
  print("🏭 Installing dependencies...")
  !wget -q --show-progress {url}
  if get_ipython().__dict__['user_ns']['_exit_code']:
    return
  with zipfile.ZipFile(name, 'r') as deps:
    deps.extractall(dst)
  !dpkg -i {dst}/*
  if get_ipython().__dict__['user_ns']['_exit_code']:
    return
  os.remove(name)
  shutil.rmtree(dst)
  console.clear()
  return True

if "step2_installed_flag" not in globals():
  if ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/deb-libs.zip", "deb-libs.zip", deps_dir):
    step2_installed_flag = True

def get_json(url):
  with urlopen(Request(url, headers={"User-Agent": user_agent})) as page:
    return json.load(page)

def filter_images(data):
  return [p["file_url"] if p["width"]*p["height"] <= max_resolution**2 else p["sample_url"]
          for p in data["post"]
          if (p["parent_id"] == 0 or include_posts_with_parent)
          and p["file_url"].endswith(supported_types)]

def download_images():
  data = get_json(url)
  count = data["@attributes"]["count"]

  if count == 0:
    print("📷 No images found")
    return

  print(f"🎯 Found {count} images")
  test_url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(tags)
  display(Markdown(f"[Click here to open in browser!]({test_url})"))
  inp = input("❓ Enter the word 'yes' if you want to proceed with the download: ")

  if inp.lower().strip() != 'yes':
    print("❌ Download cancelled")
    return

  print("📩 Grabbing image list...")

  image_urls = set()
  image_urls = image_urls.union(filter_images(data))
  for i in range(total_limit // limit):
    count -= limit
    if count <= 0:
      break
    time.sleep(0.1)
    image_urls = image_urls.union(filter_images(get_json(url+f"&pid={i+1}")))

  scrape_file = os.path.join(config_folder, f"scraped_links.txt")
  with open(scrape_file, "w") as f:
    f.write("\n".join(image_urls))

  print(f"🌐 Saved links to {scrape_file}\n🔁 Starting download to {images_folder} ...\n")
  old_img_count = len([f for f in os.listdir(images_folder) if f.endswith(supported_types)])

  os.chdir(images_folder)
  !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -i {scrape_file}

  new_img_count = len([f for f in os.listdir(images_folder) if f.endswith(supported_types)])
  print(f"\n✅ Downloaded {new_img_count - old_img_count} images.")

download_images()


In [ ]:
#@markdown ### 3️⃣ Curate your images
#@markdown We will find duplicate images with the FiftyOne AI, and mark them with `delete`. <p>
#@markdown Then, an interactive area will appear below this cell that lets you visualize all your images and manually mark with `delete` to the ones you don't like. <p>
#@markdown If the interactive area appears blank for over a minute, try enabling cookies and removing tracking protection for the Google Colab website, as they may break it.
#@markdown Regardless, you can save your changes by sending Enter in the input box above the interactive area.<p>
#@markdown This is how similar 2 images must be to be marked for deletion. I recommend 0.97 to 0.99:
similarity_threshold = 0.985 #@param {type:"number"}

import os
from google.colab import output as console

os.chdir("/content")
model_name = "clip-vit-base32-torch"
supported_types = (".png", ".jpg", ".jpeg")
img_count = len(os.listdir(images_folder))
batch_size = min(250, img_count)

if "step3_installed_flag" not in globals():
  print("🏭 Installing dependencies...\n")
  !pip install fiftyone ftfy
  if not get_ipython().__dict__['user_ns']['_exit_code']:
    console.clear()
    step3_installed_flag = True

import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from sklearn.metrics.pairwise import cosine_similarity

non_images = [f for f in os.listdir(images_folder) if not f.endswith(supported_types)]
if non_images:
  print(f"💥 Error: Found non-image file {non_images[0]} - This program doesn't allow it. Sorry!")
elif img_count == 0:
  print(f"💥 Error: No images found in {images_folder}")
else:
  print("💿 Analyzing dataset...\n")
  dataset = fo.Dataset.from_dir(images_folder, dataset_type=fo.types.ImageDirectory)
  model = foz.load_zoo_model(model_name)
  embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

  batch_embeddings = np.array_split(embeddings, batch_size)
  similarity_matrices = []
  max_size_x = max(array.shape[0] for array in batch_embeddings)
  max_size_y = max(array.shape[1] for array in batch_embeddings)

  for i, batch_embedding in enumerate(batch_embeddings):
    similarity = cosine_similarity(batch_embedding)
    #Pad 0 for np.concatenate
    padded_array = np.zeros((max_size_x, max_size_y))
    padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
    similarity_matrices.append(padded_array)

  similarity_matrix = np.concatenate(similarity_matrices, axis=0)
  similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

  similarity_matrix = cosine_similarity(embeddings)
  similarity_matrix -= np.identity(len(similarity_matrix))

  dataset.match(F("max_similarity") > similarity_threshold)
  dataset.tags = ["delete", "has_duplicates"]

  id_map = [s.id for s in dataset.select_fields(["id"])]
  samples_to_remove = set()
  samples_to_keep = set()

  for idx, sample in enumerate(dataset):
    if sample.id not in samples_to_remove:
      # Keep the first instance of two duplicates
      samples_to_keep.add(sample.id)
      
      dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
      for dup in dup_idxs:
          # We kept the first instance so remove all other duplicates
          samples_to_remove.add(id_map[dup])

      if len(dup_idxs) > 0:
          sample.tags.append("has_duplicates")
          sample.save()
    else:
      sample.tags.append("delete")
      sample.save()

  console.clear()

  sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
  for group in sidebar_groups[2:]:
    group.expanded = False
  dataset.app_config.sidebar_groups = sidebar_groups
  dataset.save()
  session = fo.launch_app(dataset)

  print("❗ Wait a minute for the session to load. If it doesn't, read above.")
  print("❗ When it's ready, you'll see a grid of your images.")
  print("❗ On the left side enable the \"delete\" label to visualize the images marked for deletion.")
  print("❗ You can mark your own images with the \"delete\" label by selecting them and pressing the tag icon at the top.")
  input("⭕ When you're done, enter something here to save your changes: ")

  session.refresh()
  fo.close_app()
  console.clear()
  print("💾 Saving...")

  kys = [s for s in dataset if "delete" in s.tags]
  dataset.remove_samples(kys)
  dataset.export(export_dir=os.path.join(images_folder, project_name), dataset_type=fo.types.ImageDirectory)
  
  temp_suffix = "_temp"
  !mv {images_folder} {images_folder}{temp_suffix}
  !mv {images_folder}{temp_suffix}/{project_name} {datasets_dir}
  !rm -r {images_folder}{temp_suffix}

  print(f"\n✅ Removed {len(kys)} images from dataset. You now have {len(os.listdir(images_folder))} images.")


In [ ]:
#@markdown ### 4️⃣ Tag your images
#@markdown We will be using AI to automatically tag your images, specifically [Waifu Diffusion](https://huggingface.co/SmilingWolf/wd-v1-4-swinv2-tagger-v2) in the case of anime and [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) in the case of photoealism.
#@markdown Giving tags/captions to your images allows for much better training. This process should take a couple minutes. <p>
method = "Photorealism captions" #@param ["Anime tags", "Photorealism captions"]
#@markdown **Anime:** The threshold is the minimum level of confidence the tagger must have in order to include a tag. Lower threshold = More tags
tag_threshold = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
#@markdown **Photorealism:** The minimum and maximum length of tokens/words in each caption.
caption_min = 10 #@param {type:"number"}
caption_max = 75 #@param {type:"number"}

%env PYTHONPATH=/env/python
import os
from google.colab import output as console
from IPython import get_ipython

os.chdir("/content")
kohya = "/content/kohya-trainer"
if not os.path.exists(kohya):
  !git clone https://github.com/Linaqruf/kohya-trainer {kohya}
  os.chdir(kohya)
  !git reset --hard 86de685a8c37e60a610d08cbece3da6b3a553bc0
  os.chdir("/content")

if "tags" in method:
  if "step4a_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip install tensorflow==2.10.1 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      console.clear()
      step4a_installed_flag = True

  print("🚶‍♂️ Launching program...\n")

  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
    {images_folder} \
    --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
    --model_dir=/content \
    --thresh={tag_threshold} \
    --batch_size=8 \
    --caption_extension=.txt \
    --force_download

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    print("removing underscores...")
    from collections import Counter
    top_tags = Counter()
    for txt in [f for f in os.listdir(images_folder) if f.endswith(".txt")]:
      with open(os.path.join(images_folder, txt), 'r') as f:
        tags = [t.strip() for t in f.read().split(",")]
        tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
      top_tags.update(tags)
      with open(os.path.join(images_folder, txt), 'w') as f:
        f.write(", ".join(tags))

    %env PYTHONPATH=/env/python
    console.clear()
    print(f"📊 Tagging complete. Here are the top 50 tags in your dataset:")
    print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))

    
else: # Photorealism
  if "step4b_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      console.clear()
      step4b_installed_flag = True

  print("🚶‍♂️ Launching program...\n")

  os.chdir(kohya)
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/make_captions.py \
    {images_folder} \
    --beam_search \
    --max_data_loader_n_workers=2 \
    --batch_size=8 \
    --min_length={caption_min} \
    --max_length={caption_max} \
    --caption_extension=.txt

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    import random
    captions = [f for f in os.listdir(images_folder) if f.endswith(".txt")]
    sample = []
    for txt in random.sample(captions, min(10, len(captions))):
      with open(os.path.join(images_folder, txt), 'r') as f:
        sample.append(f.read())

    os.chdir("/content")
    %env PYTHONPATH=/env/python
    console.clear()
    print(f"📊 Captioning complete. Here are {len(sample)} example captions from your dataset:")
    print("".join(sample))
  
  


In [ ]:
import os
#@markdown ### 5️⃣ Curate your tags
#@markdown Modify your dataset's tags. You can run this cell multiple times with different parameters. <p>

#@markdown Put an activation tag at the start of every text file. This is useful to make learning better and activate your Lora easier. Set `keep_tokens` to 1 when training.<p>
#@markdown Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.
global_activation_tag = "" #@param {type:"string"}
remove_tags = "" #@param {type:"string"}
#@markdown &nbsp;

#@markdown In this advanced section, you can search text files containing matching tags, and replace them with less/more/different tags. If you select the checkbox below, any extra tags will be put at the start of the file, letting you assign different activation tags to different parts of your dataset. Still, you may want a more advanced tool for this.
search_tags = "" #@param {type:"string"}
replace_with = "" #@param {type:"string"}
search_mode = "OR" #@param ["OR", "AND"]
new_becomes_activation_tag = False #@param {type:"boolean"}
#@markdown These may be useful sometimes. Will remove existing activation tags, be careful.
sort_alphabetically = False #@param {type:"boolean"}
remove_duplicates = False #@param {type:"boolean"}

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

for txt in [f for f in os.listdir(images_folder) if f.endswith(".txt")]:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if remove_tags:
  print(f"\n🚮 Removed {remove_count} tags.")
if search_tags:
  print(f"\n💫 Replaced in {replace_count} files.")
print("\n✅ Done!")


In [ ]:
#@markdown ### 6️⃣ Ready
#@markdown You should be ready to [train your Lora](https://colab.research.google.com/drive/1fs7oHytA4Va0IzDK-F8q_q9RIJRIh5Dv?usp=sharing)!

from IPython.display import Markdown, display
display(Markdown(f"### 🦀 [Click here to open the Lora trainer](https://colab.research.google.com/drive/1fs7oHytA4Va0IzDK-F8q_q9RIJRIh5Dv?usp=sharing)"))


## *️⃣ Extras

In [ ]:
#@markdown ### 📈 Analyze Tags
#@markdown Perhaps you need another look at your dataset.
show_top_tags = 200 #@param {type:"number"}

from collections import Counter
top_tags = Counter()

for txt in [f for f in os.listdir(images_folder) if f.endswith(".txt")]:
  with open(os.path.join(images_folder, txt), 'r') as f:
    top_tags.update([s.strip() for s in f.read().split(",")])

top_tags = Counter(top_tags)
print(f"📊 Top {show_top_tags} tags:")
for k, v in top_tags.most_common(show_top_tags):
  print(f"{k} ({v})")

In [ ]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/lora_training/datasets/example.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/lora_training/datasets/example/" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [ ]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/lora_training/datasets" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
for i, (root, dirs, files) in enumerate(os.walk(folder)):
  images = len([f for f in files if f.endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images and not captions and not others \
                    else f"{images:>4} images | {captions:>4} text files | {others:>4} other files"
pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))
